In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open('concast-3.json', 'r') as f:
    j = json.load(f)

In [3]:
j.keys()

dict_keys(['jobName', 'accountId', 'results', 'status'])

In [4]:
j['results'].keys()

dict_keys(['transcripts', 'speaker_labels', 'items', 'segments'])

In [5]:
spk_data = j['results']['speaker_labels']['segments']
dict_spk = {}
for d in spk_data:    
    st = float(d['start_time'])
    et = float(d['end_time'])
    speaker = d['speaker_label']
    
    dict_spk[st] = {'end_time': et, 'speaker': speaker}

In [6]:
dict_spk

{0.34: {'end_time': 3.67, 'speaker': 'spk_0'},
 4.27: {'end_time': 6.38, 'speaker': 'spk_0'},
 6.38: {'end_time': 7.87, 'speaker': 'spk_1'},
 8.47: {'end_time': 8.8, 'speaker': 'spk_1'},
 9.94: {'end_time': 10.59, 'speaker': 'spk_1'},
 11.19: {'end_time': 12.27, 'speaker': 'spk_1'},
 13.3: {'end_time': 13.61, 'speaker': 'spk_1'},
 15.64: {'end_time': 15.9, 'speaker': 'spk_2'},
 16.54: {'end_time': 16.89, 'speaker': 'spk_2'},
 17.45: {'end_time': 19.64, 'speaker': 'spk_2'},
 22.31: {'end_time': 25.25, 'speaker': 'spk_0'},
 25.74: {'end_time': 27.76, 'speaker': 'spk_2'},
 28.64: {'end_time': 29.07, 'speaker': 'spk_1'},
 29.07: {'end_time': 29.86, 'speaker': 'spk_2'},
 33.34: {'end_time': 34.95, 'speaker': 'spk_0'},
 35.34: {'end_time': 36.02, 'speaker': 'spk_1'},
 36.65: {'end_time': 39.84, 'speaker': 'spk_1'},
 40.47: {'end_time': 41.49, 'speaker': 'spk_1'},
 41.5: {'end_time': 41.68, 'speaker': 'spk_2'},
 42.21: {'end_time': 42.83, 'speaker': 'spk_2'},
 43.39: {'end_time': 44.65, 'spea

In [7]:
def mean_confidence(items):
    confidences = []
    for item in items:
        confidences.append(item['confidence'])
    a = np.array(confidences, dtype=np.float)
    return a.mean()

In [8]:
def choose_alt(j):
    secs = []
    phrases_selected = []
    
    for seg in j['results']['segments']:
        alternatives = seg['alternatives']
        max_conf = -1
        alt_likely = ''
        for alt in alternatives:
            items = alt['items']
            confidence = mean_confidence(items)
            if confidence>max_conf:
                alt_likely = alt
                max_conf = confidence
        s_t = float(alt_likely['items'][0]['start_time'])
        
        secs.append(s_t)
        phrases_selected.append(alt_likely['transcript'].replace(" ", "").replace('LENCE]', '...'))
    return secs, phrases_selected

In [9]:
secs, contents = choose_alt(j)

In [10]:
def who_speaks(t, dict_speaker):
    try:
        return dict_speaker[t]['speaker']
    except KeyError:
        for s_t in dict_speaker.keys():
            if t>s_t:
                e_t = dict_speaker[s_t]['end_time']
                if t<e_t:
                    return dict_speaker[s_t]['speaker']
    
                

In [11]:
def define_speakers(x, dict_):
    return dict_.get(x)

df = pd.DataFrame({'t': secs, 'content': contents})
df['speaker'] = df['t'].map(lambda t: who_speaks(t, dict_spk))

#話者dict
dict_speakers = {'spk_0':'K', 'spk_1': 'gota', 'spk_2': 'TKB84', None:'unk'}
df['speaker'] = df['speaker'].map(lambda x: define_speakers(x, dict_speakers))
old_spk = ''

list_content = []
list_contents = []
list_speakers = []
for i, row in df.iterrows():
    cur_spk = row['speaker']
    if cur_spk!=old_spk:
        list_speakers.append(old_spk)
        list_contents.append('、'.join(list_content))
        list_content = []

        
    list_content.append(row['content'])
    old_spk = cur_spk

In [12]:
def narrative(df):
    for spk, content in df[['speaker', 'content']].values:
        print(f'{spk}:\n  {content}')

In [13]:
df = pd.DataFrame({'speaker':list_speakers, 'content':list_contents})
narrative(df)

:
  
K:
  じゃあ今日はえーっと西国三回目ってことで、ゲストの方にして
gota:
  いただきましたけれど、ハワイ人、の子供を、めっちゃ正しいですよこれ、だけ
TKB84:
  ああ、このあと、自分の声を聞くってことも言っちゃったし
K:
  凄い嫌になりましたね最初
TKB84:
  いや今、言うこういうのを始めたので
gota:
  出現
TKB84:
  がないか緊張性
K:
  早くカットできるので
gota:
  冗談でしょ、あのなんか言っちゃったらと思ったらこう言ってもらえれば業界、でも安心し
TKB84:
  て、対応できますね、ここここ筆みたいな
gota:
  あの、もう最初にオフレコって言ってもらえばソレはそれでもカットする、わけです、からねどうしますかえーっと、さっき
TKB84:
  の話だけ、どこに
K:
  話した、えーっとハードルとフィットネスは心関係子というか分業のところでしたそう
gota:
  だ、僕がいない間にもそんな話だったらちょっとキャッチャーの指揮したりされ
TKB84:
  食べてみたい、まあ
K:
  まあそうですね高橋さんのなんかまず自己紹介的なところ
gota:
  からそうですねはい
TKB84:
  えハードルっていう会社で働いてるたぶん前回の、ですよね、ポッドキャストで若干言及して貰ってるハードレティ分析ソフトを使って、いる会社の、で日本の営業担当している高林と申します、え去年の十二月にジョイン
K:
  した
TKB84:
  ばかりなので、まだまだこれこそ機能、尺みたいなヘビーユーザーの方に追いつけてません!けど日々頑張ってプロダクトのことを勉強して今日本の、色んなプロチームだったり高校当たりだったりま。他のチームだったのに対し併用するような立場で動いており
gota:
  ますへと、いたします
K:
  一つ...、凄いあのーやっぱり僕らの大先輩ですから
gota:
  ちょっと大大大先輩だったらはい
TKB84:
  いやいや、いややめやめて今後
gota:
  当社が欲しい、じゃあまあ当然年齢とかじゃなくても、気持ちの面でも代々尊敬しているのではい
TKB84:
  これはだいぶあり初のヘリが入ってる感じだよ!
gota:
  ほら実際まだあのー、あれですね、あのー、これちゃんと面向かって